# A1 — Classificação de tipo de terreno com LiDAR compacto (Fácil)

Classifique o tipo de terreno (liso, cascalho, grama) a partir de pequenas características de um patch de LiDAR usando um **kernel quântico** ou **VQC**.

### Contexto
**LiDAR (Light Detection and Ranging)** emite pulsos de laser e mede o tempo de retorno para construir uma nuvem de pontos 3D ou amostras localizadas de altura/intensidade. Pequenos patches voltados ao solo em um robô móvel ajudam a antecipar tração e vibração do terreno à frente.

Em robótica de campo, saber se a superfície adiante é asfalto liso, cascalho solto ou grama influencia:
- Planejamento de movimento (slip esperado / custo energético)
- Colocação de pés ou rodas e perfis de velocidade
- Controle antecipatório (alocação de torque, ajuste de marcha)

O conjunto de dados comprime cada patch em vetores de características engenheiradas (por exemplo, estatísticas de altura local, indicadores de rugosidade). Você recebe apenas esses vetores (sem nuvem bruta) para manter a dimensionalidade baixa (d=12) e tornar codificações quânticas viáveis (≤ 10 qubits).

### Por que métodos quânticos?
Fronteiras de decisão moderadamente não lineares em baixa dimensão são um regime plausível onde mapeamentos de características (feature maps) quânticos ou circuitos variacionais enriquecem o kernel sob orçamento de qubits restrito. Objetivo: projetar tal embedding ou classificador que generalize melhor que o SVM RBF clássico em seeds ocultos.

Meta: superar a acurácia do baseline no conjunto oculto com ≤ 10 qubits e ≤ 150 passos de otimização.

### Mini‑resumo de LiDAR
**LiDAR** emite pulsos curtos e mede tempo de voo para estimar alcance $r$ e, frequentemente, registra intensidade $I$. Padrões de varredura (rotativos ou solid‑state) geram amostras esparsas da microestrutura da superfície. Para uso voltado ao solo, um pequeno patch à frente reflete micro‑geometria e material:
- Asfalto liso: baixa variância de micro‑altura, baixa rugosidade, intensidade estável.
- Cascalho solto: variância e rugosidade mais altas; jitter de intensidade por múltiplos retornos.
- Grama: variância moderada com anisotropia (estruturas de lâminas) e intensidade tipo speckle.

Neste desafio, pré‑computamos características compactas (d=12) por patch para manter qubits pequenos, preservando pistas discriminativas (médias/variâncias, proxies de rugosidade/curvatura, energia de gradiente, espalho de intensidade) — suficiente para kernels quânticos ou VQCs pequenos.

### Geração de dados e características
Geramos vetores por classe com médias e variâncias específicas e embaralhamos para balancear. Veja [`common.data_utils.terrain_dataset`](common/data_utils.py) para a construção (ordem: liso → cascalho → grama). Isso enfatiza a qualidade do embedding em vez do processamento da nuvem bruta.

## Especificação de entrada e saída
**Entrada:** `X_train (N×d)`, `y_train (N,)`, `X_test (M×d)` com d=12 nos dados fornecidos  
**Rótulos:** inteiros em {0,1,2} mapeando para classes [0=liso, 1=cascalho, 2=grama]  
**Saída:** `y_pred (M,)` classes previstas em {0,1,2}  
**Tipo de retorno:** array de inteiros (ex.: `np.int64`)

Dicas de desempenho: mantenha ≤ 10 qubits e ≤ 150 passos de otimização; evite loops pesados em Python por amostra.

## Setup

In [ ]:
import sys
import os
# Adiciona o diretório pai (qhacka-2025) ao caminho do Python
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
import numpy as np
from common import data_utils as du
from common import baselines as bl
from common import quantum_utils as qu
np.random.seed(1337)

## Baseline (referência)

In [4]:
# Dados de exemplo
Xtr, ytr = du.terrain_dataset(n=240, d=12, seed=0)
Xte, yte = du.terrain_dataset(n=120, d=12, seed=101)

# Baseline: SVM RBF
yp = bl.baseline_terrain(Xtr, ytr, Xte)
acc = (yp == yte).mean()
print(f"Acurácia do baseline: {acc:.3f}")

Acurácia do baseline: 1.000


## Sua tarefa: implementar `solve(...)`

In [ ]:
def solve(Xtr, ytr, Xte):
    
    # --- Início da Solução Kernel ---

    # 1. Importar bibliotecas
    import pennylane as qml
    from pennylane.kernels import kernel_matrix
    from sklearn.svm import SVC
    from sklearn.preprocessing import StandardScaler
    from sklearn.decomposition import PCA
    from common import quantum_utils as qu
    import numpy as np 

    # 2. Pré-processamento (Scaler + PCA)
    n_qubits = 4 
    
    scaler = StandardScaler()
    Xtr_norm = scaler.fit_transform(Xtr)
    Xte_norm = scaler.transform(Xte)

    pca = PCA(n_components=n_qubits)
    Xtr_pca = pca.fit_transform(Xtr_norm)
    Xte_pca = pca.transform(Xte_norm)

    # 3. --- PONTO DE EQUILÍBRIO (Robustez vs. Velocidade) ---
    # 10 foi rápido (7.2s) mas viciado (0.675)
    # 20 foi lento (14.3s) mas robusto (0.791)
    # 30 foi lento demais (25.6s)
    n_subset = 12 # <-- Ponto de equilíbrio
    
    np.random.seed(42) 
    subset_indices = np.random.choice(len(Xtr_pca), n_subset, replace=False)
    
    Xtr_subset = Xtr_pca[subset_indices]
    ytr_subset = ytr[subset_indices]

    # 4. Configurar o dispositivo quântico
    dev = qu.default_device(n_qubits=n_qubits) 

    # 5. CRIAR O QNODE DO KERNEL
    @qml.qnode(dev)
    def kernel_circuit(x1, x2):
        qu.feature_map(x1, wires=range(n_qubits))
        qml.adjoint(qu.feature_map)(x2, wires=range(n_qubits))
        return qml.expval(qml.Projector([0]*n_qubits, wires=range(n_qubits)))

    # 6. Calcular as Matrizes de Kernel
    kernel_train = kernel_matrix(
        Xtr_subset,
        Xtr_subset,
        kernel_circuit
    )
    
    kernel_test = kernel_matrix(
        Xte_pca,
        Xtr_subset,
        kernel_circuit
    )

    # 7. Treinar o SVM
    clf = SVC(kernel='precomputed')
    clf.fit(kernel_train, ytr_subset)

    # 8. Fazer a previsão
    y_pred = clf.predict(kernel_test)
    return y_pred
    
    # --- Fim da Solução Kernel ---

## Testes públicos (rápidos)

In [41]:
# Testes públicos
Xtr, ytr = du.terrain_dataset(n=240, d=12, seed=1)
Xte, yte = du.terrain_dataset(n=120, d=12, seed=2)
yp = solve(Xtr, ytr, Xte)
acc = (yp == yte).mean()
print("Acurácia pública:", acc)
assert yp.shape == yte.shape
assert acc >= 0.70, "Limiar público não atingido"
print("OK")

Acurácia pública: 0.925
OK


> Testes adicionais serão executados pelos organizadores com seeds/tamanhos diferentes.

In [42]:
# --- CÉLULA DE VALIDAÇÃO (seed=99) ---
print("--- Iniciando Teste de Validação (Robustedez) ---")

# Usar um seed diferente (ex: 99) para simular os testes ocultos da banca
Xtr_val, ytr_val = du.terrain_dataset(n=240, d=12, seed=99)
Xte_val, yte_val = du.terrain_dataset(n=120, d=12, seed=100)

# Rodar a mesma solução 'solve'
yp_val = solve(Xtr_val, ytr_val, Xte_val)
acc_val = (yp_val == yte_val).mean()

print("Acurácia Validação (seed=99):", acc_val)

# Pegar a acurácia do teste público (variável 'acc' da célula anterior)
try:
    print("Acurácia Pública   (seed=2):", acc) 
    print(f"Diferença: {abs(acc - acc_val):.4f}")
    assert abs(acc - acc_val) < 0.15, "Overfitting detectado! A diferença é muito alta."
    print("VEREDITO: Solução robusta, não houve overfitting.")
except NameError:
    print("Execute a célula de Teste Público primeiro para comparar.")

print("--- Fim do Teste de Validação ---")

--- Iniciando Teste de Validação (Robustedez) ---
Acurácia Validação (seed=99): 0.7916666666666666
Acurácia Pública   (seed=2): 0.925
Diferença: 0.1333
VEREDITO: Solução robusta, não houve overfitting.
--- Fim do Teste de Validação ---
